In [66]:
import asyncio
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")


#df_c4z = pd.read_csv("csv/test2.csv")
df_26nf = pd.read_csv("csv/test2.csv")
#data_c4z=data_c4z[:1000]
#data_26nf=data_26nf[:1000]

#df_index_price = pd.read_csv("2024-11-14.csv")
#df_index_price=df_index_price[:1000]


In [67]:
# #df_c4z.drop(columns=['marketId', 'subaccountId', 'executionType', 'orderHash','quantity'], inplace=True)
df_26nf.drop(columns=['marketId', 'subaccountId', 'executionType', 'orderHash'], inplace=True)

#df_26nf = df_26nf[df_26nf['orderType'].isin(['buy_po', 'sell_po'])]

#df_26nf['price'] = df_26nf['price'] * 10**12
#df_c4z['price'] = df_c4z['price'] * 10**12

In [68]:
#df_26nf

,orderType,price,quantity,updated_at,state,created_at
0,sell_po,34830000,55.81,2025-01-23 23:42:53.599,canceled,2025-01-23 23:42:46.746
1,buy_po,34630000,56.14,2025-01-23 23:42:53.599,canceled,2025-01-23 23:42:46.746
2,sell_po,34880000,55.73,2025-01-23 23:42:53.599,booked,2025-01-23 23:42:53.599
3,buy_po,34680000,56.05,2025-01-23 23:42:53.599,booked,2025-01-23 23:42:53.599
4,sell_po,34790000,55.88,2025-01-23 23:42:46.746,canceled,2025-01-23 23:42:29.153
...,...,...,...,...,...,...
2751,buy_po,36920000,55.98,2025-01-23 17:44:34.377,canceled,2025-01-23 17:44:21.716
2752,sell_po,37090000,55.73,2025-01-23 17:44:21.716,canceled,2025-01-23 17:43:15.152
2753,buy_po,36870000,56.06,2025-01-23 17:44:21.716,canceled,2025-01-23 17:43:15.152
2754,sell_po,37140000,55.65,2025-01-23 17:43:15.152,canceled,2025-01-23 17:43:03.268


In [69]:


df_transformed = df_26nf.pivot_table(
    index=["updated_at", "state", "created_at"],
    columns="orderType",
    values="price",
    aggfunc="first"
).reset_index()

print(df_transformed.columns) 
# Rename columns for clarity
#df_transformed.columns = ["updated_at", "state", "created_at", "buy_price", "sell_price"]
#df_transformed["buy"]= df_transformed["buy"]/1e6
#df_transformed["sell"]= df_transformed["sell"]/1e6
#df_transformed["buy_po"]= df_transformed["buy_po"]/1e6
#df_transformed["sell_po"]= df_transformed["sell_po"]/1e6
# Display the transformed DataFrame
df_transformed["offset"] = 100 * (df_transformed["sell_po"]-df_transformed["buy_po"])/ ((df_transformed["sell_po"]+df_transformed["buy_po"])/2)/2
df_transformed

Index(['updated_at', 'state', 'created_at', 'buy_po', 'sell_po'], dtype='object', name='orderType')


orderType,updated_at,state,created_at,buy_po,sell_po,offset
0,2025-01-23 17:43:15.152,canceled,2025-01-23 17:43:03.268,36920000.0,37140000.0,0.297056
1,2025-01-23 17:44:21.716,canceled,2025-01-23 17:43:15.152,36870000.0,37090000.0,0.297458
2,2025-01-23 17:44:34.377,canceled,2025-01-23 17:44:21.716,36920000.0,37140000.0,0.297056
3,2025-01-23 17:45:01.112,canceled,2025-01-23 17:44:34.377,36970000.0,37190000.0,0.296656
4,2025-01-23 17:45:11.194,canceled,2025-01-23 17:45:01.112,36910000.0,37130000.0,0.297137
...,...,...,...,...,...,...
1558,2025-01-23 23:42:26.336,canceled,2025-01-23 23:42:24.266,34590000.0,34790000.0,0.288268
1559,2025-01-23 23:42:29.153,canceled,2025-01-23 23:42:26.336,34540000.0,34740000.0,0.288684
1560,2025-01-23 23:42:46.746,canceled,2025-01-23 23:42:29.153,34590000.0,34790000.0,0.288268
1561,2025-01-23 23:42:53.599,booked,2025-01-23 23:42:53.599,34680000.0,34880000.0,0.287522


In [70]:
import plotly.graph_objects as go


fig = go.Figure()

# Add traces for each y-column
fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['buy_po'],
                         mode='lines', name='buy '))
fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['sell_po'],
                         mode='lines', name='sell '))
#fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['buy_po'],mode='lines', name='buy '))
#fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['sell_po'],mode='lines', name='sell '))
#Update layout for titles and axes
fig.update_layout(
    title="Offsets over Time",
    xaxis_title="Updated At",
    yaxis_title="Offset Value",
    legend_title="Offsets",
)

# Show the figure
fig.show()


In [71]:
import plotly.express as px

# Create a sample DataFrame

# Create a line plot from the DataFrame
fig = px.line(df_transformed, x='updated_at', y='offset', title='Offset from c4z order')

# Show the plot
fig.show()
